In [1]:
import pandas as pd
import numpy as np
import re, string

In [2]:
df = pd.read_csv('desc_1000.csv',delimiter='|')

In [3]:
def rem_html(desc):
    return re.sub('<[^<]+?>', '', desc)

def rem_punc(desc):
    table = str.maketrans({key:None for key in string.punctuation})
    return desc.translate(table)

def cleaner(desc):
    return rem_punc(rem_html(desc)).lower()



In [4]:
def hand_label_topics(H, vocabulary):
    '''
    Print the most influential words of each latent topic, and prompt the user
    to label each topic. The user should use their humanness to figure out what
    each latent topic is capturing.
    '''
    hand_labels = []
    for i, row in enumerate(H):
        top_five = np.argsort(row)[::-1][:20]
        print( 'topic', i)
        print( '-->', ' '.join(vocabulary[top_five]))
        label = raw_input('please label this topic: ')
        hand_labels.append(label)
        print()
    return hand_labels

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF 

In [7]:
df['desc_clean'] = np.vectorize(cleaner)(df.desc)

In [8]:
vectorizer = TfidfVectorizer()

In [9]:
X = vectorizer.fit_transform(df.desc_clean)

/Users/colinbrochard/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [10]:
# We'd like to see consistent results, so set the seed.
nmf = NMF(n_components=7, max_iter=100, random_state=12345, alpha=0.0)

# Find latent topics using our NMF model.
W = nmf.fit_transform(X)
H = nmf.components_
print('reconstruction error:', nmf.reconstruction_err_)


reconstruction error: 29.219584552004875


In [ ]:
W